# Firebase Demo

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 1000)

### establishing connection

In [2]:
def establish_creds():
    # Use a service account
    cred = credentials.Certificate('cred.json')
    firebase_admin.initialize_app(cred)
establish_creds()

In [3]:
db = firestore.client()

### read a collection

In [12]:
users_ref = db.collection("employees")
docs = users_ref.stream()

In [15]:
def print_doc(doc):
    d = doc.to_dict()
    df = pd.DataFrame([d], index=[doc.id])
    display(df)

def print_docs(docs):
    for doc in docs:
        print_doc(doc)

In [6]:
print_docs(docs)

,address,contactNumber,gender,name,role,salary,status
HcPJ9SsXZ0CneLIggMAe,1 university st.,+7222123123,Male,slava23,Doctor,36000,permanent


,address,contactNumber,gender,name,role,salary
bvKMrnpwSsBYLjR7IyNL,sdf,+72342342323,Male,Tom,Nurse,12000


,address,contractNumber,gender,name,role,salary,status
c8xwnElchX9oG4314Jrl,Innopolis,+98765432101,M,Rishat Maksudov,Doctor,18000,intern


,address,contractNumber,gender,name,role,salary,status
fNJ820rWsWpA8qMn7Tpq,Innopolis,+98765432101,M,Rishat Maksudov,Doctor,18000,intern


### read a document with a specific id

In [11]:
doc_ref = db.collection("employees").document("HcPJ9SsXZ0CneLIggMAe")
doc = doc_ref.get()
print_doc(doc)

,address,contactNumber,gender,name,role,salary,status
HcPJ9SsXZ0CneLIggMAe,1 university st.,+7222123123,Male,slava23,Doctor,36000,permanent


### insert a document

In [12]:
new_employee = {
    'address': 'Innopolis',
    'contractNumber': '+98765432101',
    'gender': 'M',
    'name': 'Rishat Maksudov',
    'role': 'Nurse',
    'salary': 12000,
}

new_em_ref = db.collection('employees').document()
new_em_ref.set(new_employee)

update_time {
  seconds: 1555932227
  nanos: 540077000
}

In [9]:
def find_rishat():
    refs = db.collection('employees').where('name', '==', 'Rishat Maksudov')
    for doc in refs.stream():
        print_doc(doc)
        break

find_rishat()

,address,contractNumber,gender,name,role,salary
c8xwnElchX9oG4314Jrl,Innopolis,+98765432101,M,Rishat Maksudov,Nurse,12000


### update the previously inserted document

In [10]:
new_em_ref.update({
    'role': 'Doctor',
    'status': 'intern',
    'salary': 18000,
})

update_time {
  seconds: 1555921627
  nanos: 574532000
}

In [11]:
find_rishat()

,address,contractNumber,gender,name,role,salary,status
c8xwnElchX9oG4314Jrl,Innopolis,+98765432101,M,Rishat Maksudov,Doctor,18000,intern


### order and limit

In [12]:
docs = db.collection('medicines').order_by('name').limit(3).stream()
print_docs(docs)

,expDate,id,name,price,quantity
3RanwQNTw2bsWDbwDaiP,22042020,3RanwQNTw2bsWDbwDaiP,Antigrippin,120,3


,expDate,id,name,price,quantity
3hkAxJaHcElcp7nuWHWQ,27122020,3hkAxJaHcElcp7nuWHWQ,Aspirin,220,15


,expDate,id,name,price,quantity
y14xgNouEm71CVMBDqLI,22072020,y14xgNouEm71CVMBDqLI,Maxicold,333,29


### perform simple queries

In [13]:
query = db.collection("medicines").where('price', '>', 120)
docs = query.stream()
print_docs(docs)

,expDate,id,name,price,quantity
3hkAxJaHcElcp7nuWHWQ,27122020,3hkAxJaHcElcp7nuWHWQ,Aspirin,220,15


,expDate,id,name,price,quantity
y14xgNouEm71CVMBDqLI,22072020,y14xgNouEm71CVMBDqLI,Maxicold,333,29


,expDate,id,name,price,quantity
BqoD26tOQJF10HMoKZlp,12312414,BqoD26tOQJF10HMoKZlp,ячсячс,2123,12316


### let's face reference fields

In [14]:
query = db.collection("treatments")
docs = query.stream()
print_docs(docs)

,bills,description,patient
MXu7wtBc7fh60DW385lT,[<google.cloud.firestore_v1.document.DocumentReference object at 0x7fb3991a4710>],some text here,<google.cloud.firestore_v1.document.DocumentReference object at 0x7fb3991a4630>


### kind of a join operation

    SELECT patient FROM patient AS pat
                   JOIN treatments AS tr ON pat=tr.patient

In [15]:
# query treatments
query = db.collection("treatments")
docs = query.stream()
for doc in docs:
    patient_ref = doc.to_dict()['patient']
    # query patients independently
    patient_doc = patient_ref.get()
    print_doc(patient_doc)

,PID,Pdetails,address,contactNumber,doctor,email,gender,name
5gHX8aaONRHNKO8HyUNB,5gHX8aaONRHNKO8HyUNB,"{'dateDischarged': 2018-03-03 13:50:09+00:00, 'dateAdmitted': 2018-02-26 12:28:48+00:00}","1744 Andrea Canyon\nChristyville, OR 31266",(385)955-2712x91780,<google.cloud.firestore_v1.document.DocumentReference object at 0x7fb3991898d0>,sharonmeyer@gmail.com,M,Francisco Williams


### perform compound queries (+contraints)
You can also chain multiple where() methods to create more specific queries (logical AND). However, to combine the equality operator (==) with a range or array-contains clause (<, <=, >, >=, or array_contains), make sure to create a composite index.

In [16]:
# P.S. it should fail
patient_ref = db.collection("patients").where('gender', '==', 'M').where('email', '<', 'f')
docs = patient_ref.stream()
print_docs(docs)

FailedPrecondition: 400 The query requires an index. You can create it here: https://console.firebase.google.com/project/dmd-hospital-system/database/firestore/indexes?create_composite=ClRwcm9qZWN0cy9kbWQtaG9zcGl0YWwtc3lzdGVtL2RhdGFiYXNlcy8oZGVmYXVsdCkvY29sbGVjdGlvbkdyb3Vwcy9wYXRpZW50cy9pbmRleGVzL18QARoKCgZnZW5kZXIQARoJCgVlbWFpbBABGgwKCF9fbmFtZV9fEAE

In [17]:
patient_ref = db.collection("patients").where('gender', '==', 'M').where('name', '<', 'F')
docs = patient_ref.stream()
print_docs(docs)

,PID,Pdetails,address,contactNumber,email,gender,name
MtwrNj4CLgnNmKqmqdBQ,MtwrNj4CLgnNmKqmqdBQ,"{'dateAdmitted': 2018-06-05 09:06:03+00:00, 'dateDischarged': 2018-06-10 13:14:30+00:00}","663 Blake Route Apt. 045\nSouth Jeanetteborough, NC 77505",525.184.2502x7055,jackgarza@hotmail.com,M,Bobby Henderson


,PID,Pdetails,address,contactNumber,email,gender,name
nezkqmtlQS8wV0yK2Djs,nezkqmtlQS8wV0yK2Djs,"{'dateDischarged': 2018-07-15 11:17:24+00:00, 'dateAdmitted': 2018-07-15 05:47:13+00:00}","314 Tracey Forks\nLake Marymouth, NE 64418",755.323.3137x885,ralphcarter@hotmail.com,M,Caleb Nguyen


,PID,Pdetails,address,contactNumber,email,gender,name
KCdtZdqXtp5d70m0Lbn5,KCdtZdqXtp5d70m0Lbn5,"{'dateAdmitted': 2018-12-26 11:18:11+00:00, 'dateDischarged': 2018-12-27 01:55:48+00:00}","1500 Smith Motorway Apt. 039\nGordonburgh, ME 91258",021-453-7270,lisavargas@hotmail.com,M,Dustin Haney


You can only perform range comparisons (<, <=, >, >=) on a single field, and you can include at most one array_contains clause in a compound query:


In [18]:
ref = db.collection("employees").where('salary', '>=', 14000).where('salary', '<=', 37000)
docs = ref.stream()
print_docs(docs)

,address,contractNumber,gender,name,role,salary,status
c8xwnElchX9oG4314Jrl,Innopolis,+98765432101,M,Rishat Maksudov,Doctor,18000,intern


,address,contractNumber,gender,name,role,salary,status
fNJ820rWsWpA8qMn7Tpq,Innopolis,+98765432101,M,Rishat Maksudov,Doctor,18000,intern


,address,contactNumber,gender,name,role,salary,status
HcPJ9SsXZ0CneLIggMAe,1 university st.,+7222123123,Male,slava23,Doctor,36000,permanent


In [19]:
# P.S. it should fail
ref = db.collection("employees").where('salary', '>=', 14000).where('name', '>=', 'T')
docs = ref.stream()
print_docs(docs)

InvalidArgument: 400 Cannot have inequality filters on multiple properties: [name, salary]

### let's make a multiple join query

    SELECT nurse FROM patients AS pat
             JOIN room_assignment AS asn ON pat=asn.patient 
             JOIN room AS r ON asn.room=r 
             JOIN employees AS nurse ON r.nurse=nurse
             WHERE pat.name='Francisco Williams'

In [19]:
def operation():
    query1 = db.collection('patients').where('name', '==', 'Francisco Williams')
    patients = query1.stream()
    for pat in patients:
        ref_line = '/patients/'+pat.id
        query2 = db.collection('room_assignment').where('patient', '==', pat.reference)
        assigns = query2.stream()
        for asn in assigns:
            room_ref = asn.to_dict()['room']
            room_doc = room_ref.get()
            nurse_ref = room_doc.to_dict()['nurse']
            nurse_doc = nurse_ref.get()
            print_doc(nurse_doc)
operation()

,address,contactNumber,gender,name,role,salary
bvKMrnpwSsBYLjR7IyNL,sdf,+72342342323,Male,Tom,Nurse,12000
